# Capítulo 8 - Regressão Linear Múltipla

No capítulo anterior vimos o caso de termos duas variáveis, uma dependente e outra independente.

Mas e se acharmos que há mais que uma variável independente que tem impacto na variável dependente?

In [ ]:
import statsmodels.formula.api as smf
import pandas as pd
import yfinance as yf
from scipy import stats

def normalize(df):
    df = df.dropna()
    return (df / df.iloc[0]) * 100

def merge_time_series(df_1, df_2, how='outer'):
    df = df_1.merge(df_2, how=how, left_index=True, right_index=True)
    return df

def download_yahoo_data(tickers, normalize_quotes=True,
                      start='1970-01-01', end='2030-12-31'):
    quotes=pd.DataFrame()
    for ticker in tickers:
        df = yf.download(ticker, start=start, end=end, progress=False)
        df = df[['Adj Close']]
        df.columns=[ticker]
        quotes = merge_time_series(quotes, df)

    quotes = quotes.ffill()

    if normalize_quotes:
        quotes = normalize(quotes)

    return quotes

def compute_t_critical_region(alpha, df):
    return stats.t.ppf(alpha,df),  stats.t.ppf(1-alpha,df)

## Equação

Passamos a ter uma regressão linear múltipla.

Se no modelo linear tinhamos como função:

$Y_i = b_0 + b_1X_i + ε_i$

No modelo múltiplo temos:

$Y_i = b_0 + b_1X1_i + b_2X_{2i} + … + b_kX_{ki} + ε_i$

## Exemplo (Exhibit 1 página 594)

Queremos analisar o bid/ask spread de uma acção. A nossa teoria é que o número de [market makers](https://www.investopedia.com/terms/m/marketmaker.asp#:~:text=The%20term%20market%20maker%20refers,in%20the%20bid%2Dask%20spread.) **e** a capitalização bolsista da empresa têm ambos impacto no bid/ask spread.

Ou seja, o nosso modelo vai ter em consideração duas variáveis independentes e uma dependente.

- Variável dependente: Bid/ask spread da acção de uma empresa
- Variável independente 1: O número de market makers
- Variável independente 2: O capitalização bolsista da empresa

A regressão resultou no seguinte quadro ANOVA:

![](https://s3.us-east-2.amazonaws.com/cbs.pg.repository/img/BidAskSpreadExample.PNG)

### Equação

Neste caso temos o modelo:

$Y_i = 1.5949 - 1.5186 X1_i - 0.3790_2X_{2i} + ε_i$

### Análise dos regressores

Na análise da equação temos de ter cuidado pois ao analisarmos o impacto de uma variável temos de ter o cuidado que tudo o resto tem de se manter constante, ou seja as outras variáveis não se mexem.

> Para aplicar este processo à nossa regressão vemos que o coeficiente da estimativa do coeficiente no logaritmo natural da capitalização de mercado é -0,3790. Portanto, o modelo prevê que um aumento de 1 no logaritmo natural da empresa a capitalização de mercado está associada a uma mudança de -0,3790 no logaritmo natural de a relação entre o spread bid-ask e o preço da ação, mantendo o logaritmo natural de o número de criadores de mercado constante.

### Significância ($F$)

O "Significance F" mostra se o nosso modelo é explicativo. No nosso exemplo o analista tinha um intervalo de confiança de 0.01, ou 1%. Um Significance F inferior diz que o nosso modelo é de facto explicativo.

Python e R mostram um p-value para cada variável independente e podemos ver imediatamente quais as variáveis que têm poder explicativo: [Multiple Regression Using Statsmodels](https://www.datarobot.com/blog/multiple-regression-using-statsmodels/) (no livro, neste exemplo eles calculam à mão).

### Significância ($r^2$)

Neste caso podemos ver no quadro o mutiple $r^2$ que no nosso caso é de 0.63 o que representa um valor significativo e nos diz que o modelo é significativo.

Normalmente encontram também um $r^2$ ajustado. (ver no link do statsmodel acima).

O que é o $r^2$?

> This represents the R-squared value, adjusted for the number of predictor variables in the model.

> This metric would be useful if we, say, fit another regression model with 10 predictors and found that the Adjusted R-squared of that model was 0.88. This would indicate that the regression model with just two predictors is better because it has a higher adjusted R-squared value.

**Fonte**: statology.org

### Variáveis que devemos tirar do modelo

Nem todos as variáveis são necessáriamente explicativas: [An introduction to Statistical Learning](https://hastie.su.domains/ISLR2/ISLRv2_website.pdf) (página 84).

Quando isso acontece devemos tirar as variáveis do modelo e se estamos correctos o r^2 ajustado no deverá descer e poderá até aumentar. Um modelo mais simples é superior ao mais complesxo. Não devemos adicionar variáveis só porque sim.

## Pressupostos do modelo de Regressão Linear Múltipla

 - 1) A relação entre a variável deprendente e as variáveis independentes é linear;
 - 2) As variáveis independentes não são aleatórias, isto é são fixas e conhecidas. Não existe uma relação linear exacta entre as variáveis independentes;
 - 3) O valor esperado do termo de erro ε é 0;
 - 4) A variância do termo de erro é a mesma para todas as observações;
 - 5) As observações do termo de erro não são correlacionadas;
 - 6) O termo de erro é normalmente distribuído;

 Reparem que quase todas os pressupostos são semelhantes ao do modelo simples, sendo que adicionamos o pressuposto 2. A violação do pressuposto 2 resulta em multicolinearidade (exemplo do "An introduction to Statistical Learning" em que newspaper foi tirado por ser uma relação muito próxima com rádio).

## Exemplo (Exhibit 3 pag. 602)

Suponhamos que estamos a considerar um investimento no fundo Fidelity Select Technology Portfolio (FSPTX), um fundo de investimento dos EUA especializado em ações de tecnologia. Queremos saber se o fundo se comporta mais como um fundo de crescimento (growth) ou um fundo de grande valor (value).

Vamos por isso comparar os retornos mensais de FSPTX com o índice S&P Growth 500 e com o índice S&P 500 Value.

Este é o output do software:

![](https://s3.us-east-2.amazonaws.com/cbs.pg.repository/img/FSPTX.PNG)

Que é que podemos ver logo assim de relance?

Significância do modelo? Coeficiente de determinação? Relação com o S&P 500 Growth e Value, positiva ou negativa?

### Exercício 1

Num determinado mês há uma valorização do indíce growth de 1% e uma desvalorização do indíce Value em 2%. Qual o valor da variação que o nosso modelo estima para o fundo FSPTX?

In [ ]:
# Yt = b0 + b1X1t + b2X2t + εt

0.0011 + (1.5850*0.01) + (-0.3902*-0.02)

0.024753999999999998

Contudo queremos saber se o indíce Growth tem um impacto no modelo, ou seja, se é individualmente significativo, com um grau de confiança de 5%

Neste exemplo não temos o p-value mas temos a t-Statistic para cada uma das duas variáveis independentes.

Podemos calcular a t-Statistic a partir da qual rejeitamos a hipotese nula.

Neste caso a t-Statistic é:

In [ ]:
compute_t_critical_region(0.025, 57)

(-2.002465458054599, 2.0024654580545986)

Como o valor absoluto da t-Statistic para o Value é de 2.93, que sendo maior que 2 podemos dizer que é significativo.

O que acontece se aumentarmos o grau de confiança para 1%?

### Fazendo o exercício em python

Demonstração realista do exercício acima. Não é preciso saber mas precisam de saber intrepretar.

In [ ]:
# ANALIZAR QUEBRA EM 2020

tickers = ['FSPTX', 'IVW', 'IVE']
nomes = ['Ftech', 'Growth', 'Value']

quotes = download_yahoo_data(tickers, nomes).resample('BM').last().pct_change().dropna()

quotes = quotes['2000':'2020']

#create dataset
df = pd.DataFrame({'Ftech': list(quotes['FSPTX']),
                   'Growth': list(quotes['IVW']),
                   'Value': list(quotes['IVE'])
                   })

# As variáveis independentes do modelo
#fit regression model
fit = smf.ols('Ftech ~ Growth+Value', data=df).fit()

#view model summary
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                  Ftech   R-squared:                       0.798
Model:                            OLS   Adj. R-squared:                  0.797
Method:                 Least Squares   F-statistic:                     483.2
Date:                Fri, 25 Nov 2022   Prob (F-statistic):           1.40e-85
Time:                        22:55:27   Log-Likelihood:                 479.35
No. Observations:                 247   AIC:                            -952.7
Df Residuals:                     244   BIC:                            -942.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0007      0.002     -0.323      0.7

## Análise de qualidade do modelo

### Estatística F

Como já vimos em cima o test F diz-nos se o modelo como um todo é significativo mas não nos diz se cada variável é significativa.

Temos também interessantes interacções entre a validade individual e colectiva das variáveis independentes

- Se nenhuma das variáveis ​​independentes num modelo de regressão for significativa, o modelo como um todo também não o é;

- Numa regressão múltipla, no entanto, não podemos apenas testar individualmente todas as variáveis independentes, porque os testes individuais não têm em conta os efeitos das interações entre as variáveis ​​independentes. Ou seja, podem haver variáveis independentes que individualmente são significativas mas colectivamente não acrescentam capacidade explicativa ao modelo.

- A estatística $F$ mede o quanto as variáveis independentes, como um todo, têm boa capacidade explicativa da variável dependente;

- A estatística $T$ mede o quanto um variável independente têm capacidade explicativa sobre a variável dependente. Um análise individual de relevância não é suficiente para extrapolar para relevância num modelo múltiplo.

### $r^2$ Ajustado

Em modelos de regressão múltipla não usamos o $r^2$ normal mas sim o $r^2$ ajustado. Chama-se ajustado porque tem em conta o número de variáveis independentes. Assim, ao adicionarmos uma variável a um modelo se ela não tiver capacidade explicativa estaremos a diminuir o $r^2$ ajustado enquanto o $r^2$ fica estável.

## Caso real. Decomposição de um ETF

https://deepnote.com/@luisilva/IWDA-vs-Smart-Beta-62f360bf-772a-4260-9112-a75b9ae652ba

## Violação dos pressupostos

### Existência de heterocedasticidade

![](https://s3.us-east-2.amazonaws.com/cbs.pg.repository/img/Homoskedasticity.PNG)

![](https://s3.us-east-2.amazonaws.com/cbs.pg.repository/img/Heteroskedasticity.PNG)

> Quando os erros são heterocedásticos, o teste F para a significância geral da regressão não é confiável. Isso ocorre porque o erro médio quadrado é um estimador enviesado do verdadeiro variância populacional dada a heteroscedasticidade.

> Além disso, testes t para a significância de
coeficientes de regressão individuais não são confiáveis ​​porque a heteroscedasticidade introduz envisamentos nos estimadores do erro padrão dos coeficientes de regressão. Se uma regressão mostra heterocedasticidade significativa, os erros padrão e estatísticas de teste dados por
software de regressão estarão incorretos, a menos que sejam ajustados para heterocedasticidade.

#### Podemos testar heterocedasticidade?

Sim, com o teste de Breush and Pagan. Vamos aproveitar para fazer uma regressão linear multipla em python ao mesmo tempo que fazemos um teste de Breush and Pagan.

In [ ]:
import numpy as np
import pandas as pd

#create dataset
df = pd.DataFrame({'rating': [90, 85, 82, 88, 94, 90, 76, 75, 87, 86],
                   'points': [25, 20, 14, 16, 27, 20, 12, 15, 14, 19],
                   'assists': [5, 7, 7, 8, 5, 7, 6, 9, 9, 5],
                   'rebounds': [11, 8, 10, 6, 6, 9, 6, 10, 10, 7]})

#view dataset
df

,rating,points,assists,rebounds
0,90,25,5,11
1,85,20,7,8
2,82,14,7,10
3,88,16,8,6
4,94,27,5,6
5,90,20,7,9
6,76,12,6,6
7,75,15,9,10
8,87,14,9,10
9,86,19,5,7


In [ ]:
import statsmodels.formula.api as smf

# TESTAR TIRAR UMA OU OUTRA VARIÁVEL E VER O R^R, R^2 AJUSTADO E O F E NO PROB
#fit regression model
fit = smf.ols('rating ~ points+assists+rebounds', data=df).fit()

#view model summary
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.623
Model:                            OLS   Adj. R-squared:                  0.434
Method:                 Least Squares   F-statistic:                     3.299
Date:                Wed, 23 Nov 2022   Prob (F-statistic):             0.0995
Time:                        17:16:11   Log-Likelihood:                -26.862
No. Observations:                  10   AIC:                             61.72
Df Residuals:                       6   BIC:                             62.93
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     62.4716     14.588      4.282      0.0

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  "anyway, n=%i" % int(n))


In [ ]:
from statsmodels.compat import lzip
import statsmodels.stats.api as sms

#perform Bresuch-Pagan test
names = ['Lagrange multiplier statistic', 'p-value',
        'f-value', 'f p-value']
test = sms.het_breuschpagan(fit.resid, fit.model.exog)

lzip(names, test)

[('Lagrange multiplier statistic', 6.003951995818433),
 ('p-value', 0.11141811013399583),
 ('f-value', 3.004944880309618),
 ('f p-value', 0.11663863538255281)]

#### Corrigir heterocedasticidade

heterocedasticidade

In [ ]:
import statsmodels.formula.api as smf

# SE O MODELO SOFRESSE DE HETEROCEDASTICIDADE (que não é o caso)
#fit regression model
fit = smf.ols('rating ~ points+assists+rebounds', data=df).fit(cov_type = "HC3")

#view model summary
print(fit.summary())

                            OLS Regression Results                            
Dep. Variable:                 rating   R-squared:                       0.623
Model:                            OLS   Adj. R-squared:                  0.434
Method:                 Least Squares   F-statistic:                     3.414
Date:                Wed, 23 Nov 2022   Prob (F-statistic):             0.0937
Time:                        17:16:11   Log-Likelihood:                -26.862
No. Observations:                  10   AIC:                             61.72
Df Residuals:                       6   BIC:                             62.93
Df Model:                           3                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     62.4716     21.501      2.906      0.0

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:1542: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  "anyway, n=%i" % int(n))


### Correlação em série

Um problema mais comum - e potencialmente mais sério - do que a violação do pressuposto de homoscedasticidade é a violação do pressuposto de que os erros de regressão não estão correlacionados entre as observações.

Tentar explicar uma relação financeira específica ao longo de vários períodos é arriscado porque erros em modelos de regressão financeira são muitas vezes correlacionados ao longo do tempo.

Quando os erros de regressão são correlacionados entre as observações, dizemos que eles são correlacionados (ou autocorrelacionados). A correlação em série geralmente surge em regressões de séries temporais


Tal como acontece com a heteroscedasticidade, o principal problema causado pela correlação em série numa regressão linear é uma estimativa incorreta dos erros padrão do coeficiente de egressão computados por pacotes de software estatístico. Desde que nenhuma das variáveis ​​independentes seja um valor defasado da variável dependente (um valor da variável dependente
de um período anterior), então os próprios parâmetros estimados serão consistentes e não precisam ser ajustados para os efeitos da correlação em série. Se, porém, uma das variáveis ​​independentes é um valor defasado da variável dependente regressão de efeito - então a correlação emsérie no termo de erro fará com que todos os parâmetros estimados pela regressão linear sejam inconsistentes e não serão estimativas válidas dos verdadeiros parâmetros.

#### Testar correlação em série

Embora haja muitos testes para analisar se o nosso modelo sofre de correlação em série a mais comum e que é output padrão de software estatístico é o teste de Durbin-Watson.

Se o nosso modelo **não** sofrer de correlação em série o valor de DW vai ser próximo de 2 (ver acima análise do fundo tecnológico). Se DW variar muito (para cima ou para baixo) é provável que o nosso modelo sofra de correlação em série e mais testes serão necessários para confirmarmos (fazemos um teste de hipótese que o valor de DW é estatisticamente diferente de 2, ao nível de confiança desejado).

A diferença num modelo destes e num teste de autocorrelação [ACF](https://s3.us-east-2.amazonaws.com/cbs.pg.repository/img/ACF.png) que vimos no capítulo anterior é que agora não estamos apenas a ver autocorrelação de uma só variável , mas também de ligações e autocorrelações entre  as diferentes variáveis.

#### Corrigir correlação em série

O método para corrigir correlação em série é semelhante que para corrigir heterocedasticidade. Usar "erros padrões robustos" no modelo usando a opção *cov_type = "HAC"* no software.

A vantagem é que usando HAC obtemos valores robustos quer para correlação em série como para heterocedasticidade.

### Multicolinearidade

#### Impacto da multicolinearidade

Multicolinearidade acontece occore quando dois ou mais variáveis independentes (ou combinação entre elas) são altamente correlacionadas.

O que queremos dizer com isso?

No modelo em qua analisamos o fundo tecnológico temos como variáveis independentes o S&P 500 Growth e o S&P500 Value. O que acham que acontece se adicionarmos o S&P500?

![](https://s3.us-east-2.amazonaws.com/cbs.pg.repository/img/Multicolinearidade1.PNG)

![](https://s3.us-east-2.amazonaws.com/cbs.pg.repository/img/Multicolinearidade2.PNG)

![](https://s3.us-east-2.amazonaws.com/cbs.pg.repository/img/Multicolinearidade3.PNG)

Quais foram as consequências? Porque é que aconteceu?

#### Como corrigir multicolinearidade?

A forma mais simples é mesmo tirar a variável independente que está a criar o problema.

### Resumo

![](https://s3.us-east-2.amazonaws.com/cbs.pg.repository/img/resumo_errors.PNG)

## Princípios de especificação do modelo

**1) O modelo deve ser fundamentado num raciocínio económico e/ou financeiro que faça sentido.**

Deveríamos ser capazes de perceber e explicar o raciocínio económico por detrás da escolha das variáveis e oraciocínio deve fazer sentido. Quando esta condição é satisfeita, aumentamos as nossas hipóteses de que o modelo terá valor preditivo com novos dados.

Esta abordagem contrasta com o processo de seleção de variáveis conhecido como data mining. Em data mining, o investigador desenvolve essencialmente um modelo que explora ao máximo as características de um conjunto de dados específico. Capacidade preditiva é limitada devido ao modelo ser demasiado ajustado aos dados do passado.

**2) A forma funcional escolhida para as variáveis na regressão deve ser apropriada dada a natureza das variáveis.**

**3) O modelo deve ser parcimonioso** .

Neste contexto, “parcimonioso” significa realizando muito com pouco. Devemos esperar que cada variável incluída na regressão desempenhe um papel essencial.

**4) O modelo deve ser examinado quanto a violações das suposições de regressão antes sendo aceito.**

**5) O modelo deve ser testado e considerado útil fora da amostra antes de ser
aceite**

Um exemplo pode ser uma optimização de carteira. É noormal deixarmos os 2 ou 3 anos mais recentes fora da optimização e depois analisarmos se a optimização funciona fora da amostra e se a optimização não perdeu qualidade.

O mesmo para testar a capacidade preditiva de modelos. Se temos dados desde 2000 até 2022 usamos 2000 a 2019 para fazer o modelo. Devemos depois testa-lo no período 2019 a 2022. Embora para nós 2019 a 2022 seja passado a verdade é que para o modelo é futuro.



## Erros de especificação do modelo

- Variáveis omitidas (propositadamente ou não);
- Escala errada na variável (podemos ter de usar o log para o modelo fazer sentido como no caso do GDP e esperança média de vida);
- Dados inadequados. A amostra pode não ser a melhor ou relevante (erro de capítulo 5);

## Erros de especificação particular de séries temporais

- Incluindo variáveis dependentes defasadas como variáveis independentes em regressões com erros correlacionados serialmente;
- Inclusão de uma função de uma variável dependente como variável independente, às vezes como resultado da datação incorreta das variáveis;
- Variáveis independentes que são medidas com erro;
- Uso de variáveis não estacionárias.

De longe, a fonte mais frequente de especificação incorreta em regressões lineares que usam séries temporais de duas ou mais variáveis diferentes é uso de variáveis não estacionárias. Muito grosseiramente, não estacionariedade significa que as propriedades de uma variável, como média e variância, são não constante ao longo do tempo.

Por exemplo usar as cotações dos fundos em análises, em vez de usar os retornos.

## Modelos com variáveis dependentes qualitativas (ou categóricas)

O que são variáveis categóricas?
- O rating de uma obrigação por exemplo. AAA, AA, BB, B, C etc etc;
- Uma empresa ir à falência;
- Um cliente de um banco que vai falar o pagamento de um cartão de crédito.

Ao contrário da com a regressão linear "normal", a variável dependente aqui não é contínua mas sim discreta. E o resultado que pretendemos analisar é binário. O cliente ou falha o pagamento do cartão de crédito ou não.

Para estimar um modelo desse género, infelizmente, a regressão linear não é o melhor modelo estatístico.

Temos, por isso, que transformar o nosso modelo e a transformação mais comum é usar uma Regressão logística.

Modelos de regressão logística são usados ​​para estimar a probabilidade de um resultado discreto tendo em consideração os valores das variáveis ​​independentes usadas para explicar esse resultado.

![](https://s3.us-east-2.amazonaws.com/cbs.pg.repository/img/Logit.PNG)

### Problema e resolução (Example 3 pag. 647)

Um modelo para estudar o que influência a atribuição de um prémio a um CEO resultou neste output (prémio não monetário mas mais estilo CEO do ano. Ou se ganha ou não).

![](https://s3.us-east-2.amazonaws.com/cbs.pg.repository/img/CEOAwards.PNG)

As variáveis independentes incluem
- BOOK-TO-MARKET (rácio do valor contabilístico em relação ao de mercado);
- LNSIZE (o logaritmo natural do valor de mercado do
patrimônio da empresa);
- RETURN-1TO3, RETURN-4TO6, RETURN-7TO12 (total
retorno durante os meses 1–3, 4–6 e 7–12 antes do mês de premiação, respectivamente);
- LNTENURE (o log natural do mandato do CEO com uma empresa em número
de anos);
- FEMALE (uma variável dummy que assume o valor 1 se o CEO for
uma mulher).



**Conceito** - Variável Dummy. Podemos ter no nosso modelo variáveis independentes binárias, ou seja que tomam o valor 0 ou 1. Como neste caso o caso de a CEO ser uma mulher ou não. A leitura do output não é diferente de uma variável contínua.

**Questões:**

**1 -** Qual das opções a seguir é a razão para Kim Dalton escolher um logit
regressão para sua análise?
- **a)** AWARD é uma variável binária;
- **b)** FEMALE é uma variável binária;
- **c)** Duas variáveis binárias estão no modelo.

In [ ]:
#@title
'''
A resposta correcta é a alínea A. Não interessa se há uma variável binárias nas variáveis independentes. O que interessa é se a variável dependente é binária ou não.

'''

**2-** Que empresas são mais prováveis que tenham o CEO vencedor?

- **a)** Grandes empresas com alto índice book-to-market e cujas acções
obtiveram altos retornos nos meses 7–12 antes do mês da atribuição do prémio;
- **b)** Grandes empresas com baixo índice book-to-market e cujas acções
obtiveram altos retornos nos meses 4-6 antes do mês da atribuição do prémio;
- **c)** Grandes empresas com baixo índice book-to-market e cujas acções
obtiveram altos retornos nos meses 7–12 antes do mês da atribuição do prémio.

In [ ]:
'''
O book to market tem uma relação negativa com a atribuição do prémio (coeficiente −0.0618) logo a alína a) está excluída.

Agora só temos de ver qual a variável, entre RETURN-4TO6 RETURN-7TO12, que contibui mais para a atribuição do prémio. Neste caso é a RETURN-7TO12 com um coeficiente de 0.9345.

A resposta correcta é, por isso a alínea c).

'''

**3 -** Qual dos seguintes tipos de CEOs tem maior probabilidade de ganhar um prêmio?

- **a)** Mulheres com um longo mandato na empresa.
- **b)** Mulheres com um curto mandato na empresa.
- **c)** Homens com um longo mandato na empresa.

In [ ]:
'''
Ao analisarmos a variável independente LNTENURE podemos ver que CEOs que estão há mais tempo no seu mandato têm mais probabilidade de ganhar. Eliminamos a hipótese b).

A comparação fica entre mulher ou homem com mandato longo. Podemos ver que FEMALE tem uma contribuição positiva face a não FEMALE, logo a resposta correcta é a a)
'''

**4 -** Consideremos uma empresa para a qual as probabilidades de log de seu CEO ganhar um prêmio
com base no modelo de regressão estimado é -2,3085. O CEO
da empresa é um homem. Qual seria a probabilidade logarítmica de o CEO ganhar um prêmio se o CEO fosse uma mulher, matendo todas as outras variáveis constantes?
- **a)** −4.0154
- **b)** −3.1185
- **c)** −1.4985

In [ ]:
'''
-2.3085 + 0.8100 = -1.4985
'''

**5** - Quais são as odds do CEO masculino mencionado na pergunta anterior
ganhar o prémio?

- **a)** 0.0807
- **b)** 0.0994
- **c)** 0.2235

**Conceito** - Odds

Odds fornecem uma medida da probabilidade de um determinado resultado. São calculadas como a razão entre o número de eventos que produzem esse resultado e o número de eventos que não produz o resultado.

In [ ]:
'''
O resultado do modelo foi de -2.3085. Recordando que o modelo logit é um modelo de log-odds só temos de elevar o valor ao e de euler para obter as odd "normais".

e** -2.3085 =  0.0994
'''

# import math
# math.e** -2.3085

**6** - Supondo que as odds/hipóteses do CEO masculino ganhar o prêmio são de 0.0994, qual é a probabilidade de o CEO ganhar o prêmio?
- **a)** 9,04%
- **b)** 9,94%
- **c)** 18,27%

**Conceito** - Para converter de hipóteses para probabilidade, dividimos as hipóteses por odds + 1

In [ ]:
# 0.0994 / (1+0.0994) = 0.0904

**7 -** What is the ratio of odds that a female CEO wins an award to the odds
that a male CEO wins an award?
- **a)** 0.0807
- **b)** 0.4449
- **c)** 2.2479

In [ ]:
'''
math.e** -1.4985 = 0.22346
0.22346 / 0.0994 = 2.25

O rácio é de 2.25, ou seja as hipóteses de uma CEO feminina ganhar o prémio são cerca de 2.25 vezes maior do que a do CEO masculino
'''

**8 -** Ao estimar o modelo de regressão logit, usamos retornos
expressos em frações. Por exemplo, um retorno de 10% é inserido como 0,10.
Portanto, uma unidade é 1 ou 100%. Considere a empresa com um CEO masculino
discutido anteriormente. Para esta empresa, o retorno total durante os meses 7 a 12 antes do mês da atribuição do prémio era de 11%. Sabemos que as log odds de seu CEO ganhar o prêmio com base no modelo de regressão estimado para
−2.3085. Qual seriam as log odds do seu CEO ganhar o prêmio se o
o retorno total durante os meses 7 a 12 antes do mês de premiação foi de 12%?

In [ ]:
'''
valor inicial + diferencial no variável * coeficiente da variável
−2.3085 + 0.01*0.9345=-2.299155





  
'''